<a href="https://colab.research.google.com/github/cyamize/dacon-winning-data/blob/main/%EC%9B%90%EC%9E%90%EB%A0%A5%EB%B0%9C%EC%A0%84%EC%86%8C_%EC%83%81%ED%83%9C_%ED%8C%90%EB%8B%A8_%EB%8D%B0%EC%9D%B4%EC%BD%98_%ED%81%B4%EB%A1%A0%EC%BD%94%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

훈련했을 때 학습 데이터에 과도하게 초점이 맞춰 머신이 훈련 될 수 있어 과적합 우려가 있다.

과적합을 막기위해 교차검증을 사용하는데, 이 때 학습세트와 검증 세트를 나눠 반복해서 검증한다. 이걸 k값 만큼의 폴드세트에 k번의 학습과 검증을 k번 평가하는것이 k-fold 교차검증이다.

또한 lightgbm은 트리 기반의 학습 알고리즘으로 데이터가 클 때 속도가 빠르고 gpu로 처리가 가능한 방법이다.

# Library & Data

In [1]:
from sklearn.model_selection import KFold
import lightgbm

In [ ]:
import multiprocessing # 여러 개의 cpu에게 작업을 분산시키는 역할
from multiprocessing import Pool
from functools import partial # 함수가 받는 인자들 중 몇개를 고정 시켜서 새롭게 파생된 함수를 형성하는 역할
from data_loader_v2 import data_loader_v2 # 자체적으로 만든 data loader version_v2
import os
import pandas as pd
import numpy as np
import joblib # 텍스트 상태의 데이터가 아닌 파이썬 객체 자체를 파일로 저장
train_folder = 'data/train/'
test_folder = 'data/test/'
train_label_path = '/data/train_label.csv'

# 데이터 전처리

In [ ]:
train_list = os.listdir(train_folder)
test_list = os.listdir(test_folder)
train_label = pd.read_csv(train_label_path, index_col=0)

# 모든 csv 파일의 상태_B로 변화는 시점이 같다고 가정
# 그러나, 개별 CSV파일의 상태_B로 변화는 시점은 상이할 수 있다
def data_loader_all_v2(func, files, folder ='', train_label=None, event_time=10,nrows=60):
  func_fixed = partial(func, folder=folder, train_label=train_label, event_time=event_time, nrows=nrows)
  if __name__ == '__main__':
    pool = Pool(processes = multiprocessing.cpu_count())
    df_list = list(pool.imap(func_fixed,files))
    pool.close()
    pool.join()
  combined_df = pd.concat(df_list)

  return combined_df                       

In [ ]:
train = data_loader_all_v2(data_loader_v2, train_list, folder=train_folder, train_label=train_label, event_time=10, nrows=60)
test = data_loader_all_v2(data_loader_v2,test_list, folder=test_folder, train_label=None, event_time=20, nrows=60)

y = train['label']
train.drop('label', axis=1,inplace=True)

# 모델 학습, 검증, 저장

In [ ]:
parms = {
    'learning_rate' : 0.06,
    'num_leaves' : 400,
    'n_estimators' : 300,
    'max_depth': -1,
    'min_child_weight' : 3, 
    'subsample' : 0.8,
    'colsample_bytree' : 0.5,
    'objective' : 'multiclass',
    'n_jobs': -1
}

In [ ]:
# 4FOLD, 3SEED ENSEMBLE
# 총 12개의 모델을 평균내어 예측한다

lucky_seed=[4885,1992,1022]

for num,rs in enumerate(lucky_seed):

  kfold = KFold(n_splits=4, random_state = rs,shuffle = True)

  # dacon code
  cv=np.zeros((train.shape[0],198))

  for n, (train_idx, validation_idx) in enumerate(kfold.split(train)):

    x_train, x_validation = train.iloc[train_idx], train.iloc[validation_idx]
    y_train, y_validation = y.iloc[train_idx], y.loc[validation_idx]

    model = lightgbm.LGBMClassifier(**parms, random_state=rs)

    model.fit(x_train, y_train, eval_set = [(x_validation, y_validation)], early_stopping_rounds=30,
              verbose = 100)
    joblib.dump(model, '../2_Code_pred/%s_fold_model_%s.pkl'%(n,rs))

    # CROSS-VALIDATION , EVALUATE CV
    cv[validation_idx,:] = model.predict_proba(x_validation)

In [ ]:
# MODEL LOAD & TEST PREDICT
# 12 MODELS 평균 사용
models = os.listdir('../2_Code_pred/')
models_list = [x for x in models if x.endswith(".pkl")]
assert len(models_list) ==12
temp_predictions = np.zeros((test.shape[0],198))

for m in models_list:
    model = joblib.load('../2_Code_pred/'+m)
    predict_proba = model.predict_proba(test)
    temp_predictions += predict_proba/12

In [ ]:
# dacon code
submission = pd.DataFrame(data=np.zeros((test.shape[0],198)))
submission.index = test.index 
submission.index.name = 'id'
submission+=temp_predictions

submission = submission.sort_index()
submission = submission.groupby('id').mean()
submission.to_csv('submission.csv', index=True)

# 결과

### 데이터 전처리
PCA, Feature 정규화, Min-Max Scaling은 성능 향상에 도움이 되지 않음 Object와 NAN 값을 0으로 바꾸어 주는 전처리만 진행

### 모델 학습 검증

- Lgbm 모델 선택

Random Forest, Xgboost, LightGBM 모델 비교 결과 LGBM의 성능이 가장 좋았다.

- K-fold & Random seed를 사용한 모델 하이퍼 파라미터 튜닝

Robust 한 모델을 만들기 위해 4Kfold * 3seed 총 12개의 모델을 만듬 Early stopping 값을 작게 설정하여 over-fitting 방지 min_child_weight 값을 CV를 통해 최적화 하여 over-fitting 방지 Soft-voting 예측 방법 선택

- Soft-voting 예측

예측 시 Hard-voting 방식과 Probability를 평균내는 Soft-voting 방식을 실험 evaluation metric이 log-loss였기 때문에 probability를 평균내는 방식의 성능이 좋았음 12개의 모델의 예측을 평균 하는 방식으로 최종 결과물 제출

In [ ]:
#